In [1]:
import requests
from requests import Response
from pydantic import RootModel,BaseModel,Field,field_validator
from datetime import datetime
import pandas as pd 

class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')
    時間:datetime = Field(alias='mday')

    @field_validator('可借數量','可還數量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value
    
class Youbike(RootModel):
    root:list[Site]

try:
    youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
    youbike_url.raise_for_status()
except Exception as e:
    print(e)
else:
    data = Youbike.model_validate_json(youbike_url.text)
    all_sites = data.model_dump()

df = pd.DataFrame(all_sites)
df


,行政區域,總數量,可借數量,可還數量,時間
0,大安區,28,0,28,2024-07-01 20:16:20
1,大安區,21,2,19,2024-07-01 20:16:20
2,大安區,16,6,10,2024-07-01 20:11:20
3,大安區,11,1,10,2024-07-01 20:12:20
4,大安區,16,1,15,2024-07-01 20:12:20
...,...,...,...,...,...
1424,臺大公館校區,30,0,29,2024-07-01 20:03:15
1425,臺大公館校區,20,1,19,2024-07-01 20:04:22
1426,臺大公館校區,24,8,15,2024-07-01 20:14:20
1427,臺大公館校區,40,29,2,2024-07-01 20:15:22


In [5]:
bins = [0,1,4,7,11,100]
labels = ['0輛','1~3輛','4~6輛','7~10輛','10輛以上']

df['可借'] = pd.cut(df['可借數量'],bins=bins,right=False,labels=labels)
df

,行政區域,總數量,可借數量,可還數量,時間,可借
0,大安區,28,0,28,2024-07-01 20:16:20,0輛
1,大安區,21,2,19,2024-07-01 20:16:20,1~3輛
2,大安區,16,6,10,2024-07-01 20:11:20,4~6輛
3,大安區,11,1,10,2024-07-01 20:12:20,1~3輛
4,大安區,16,1,15,2024-07-01 20:12:20,1~3輛
...,...,...,...,...,...,...
1424,臺大公館校區,30,0,29,2024-07-01 20:03:15,0輛
1425,臺大公館校區,20,1,19,2024-07-01 20:04:22,1~3輛
1426,臺大公館校區,24,8,15,2024-07-01 20:14:20,7~10輛
1427,臺大公館校區,40,29,2,2024-07-01 20:15:22,10輛以上
